In [48]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
import re

# Assignment - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.


In [49]:
df = pd.read_csv('university_towns.txt', names = ['Text'])
df.head()

,Text
0,Alabama[edit]
1,Auburn (Auburn University)[1]
2,Florence (University of North Alabama)
3,Jacksonville (Jacksonville State University)[2]
4,Livingston (University of West Alabama)[2]


In [50]:
df2 = pd.read_csv('City_Zhvi_AllHomes.csv', encoding='cp1252')
df2.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2018-07,2018-08,2018-09,2018-10,2018-11,2018-12,2019-01,2019-02,2019-03,2019-04
0,6181,New York,NY,New York-Newark-Jersey City,Queens County,1,169300.0,169800.0,170300.0,170600.0,...,670100,670200,672900,677100,680100,680400,679300,679400,678900,677000
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles County,2,157700.0,157600.0,157500.0,157300.0,...,676900,678900,682200,686100,688800,690300,691500,691100,688700,686100
2,39051,Houston,TX,Houston-The Woodlands-Sugar Land,Harris County,3,86500.0,86600.0,86600.0,86700.0,...,180000,181100,181800,182300,183200,184500,185800,187200,187200,186000
3,17426,Chicago,IL,Chicago-Naperville-Elgin,Cook County,4,114000.0,115100.0,116100.0,116600.0,...,227200,227100,227300,227600,227800,228200,228500,229400,230000,229400
4,6915,San Antonio,TX,San Antonio-New Braunfels,Bexar County,5,86600.0,86400.0,86300.0,86300.0,...,169200,170300,171500,172700,173900,175000,175900,176500,176600,176100


In [51]:
df3 = pd.read_excel('gdplev.xls')
df3.head(10)

,"Current-Dollar and ""Real"" Gross Domestic Product",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,2016-09-29 00:00:00
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Annual,NaN,NaN,NaN,Quarterly,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,(Seasonally adjusted annual rates),NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN,NaN,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1929,104.6,1056.6,NaN,1947q1,243.1,1934.5,NaN
8,1930,92.2,966.7,NaN,1947q2,246.3,1932.3,NaN
9,1931,77.4,904.8,NaN,1947q3,250.1,1930.3,NaN


In [52]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [53]:
#def f1(row, data, param):
#    if param['state'] == '' or row[0].find('edit') > 0:
#        param['state'] = row[0].split('[')[0]
#    else:
#        town = row[0].split('(')[0].strip()
#        data.append([param['state'], town])
        
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ],
    columns=["State", "RegionName"]  )

    The following cleaning needs to be done:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Remove newline character '\n'. '''

    univtown_list = []

    with open('university_towns.txt') as univtown:
        for line in univtown:
            if 'edit' in line:
                town = line.split('[')[0].strip()
            else:
                univtown_list.append((town, line.split('(')[0].strip()))

    univtown_df = pd.DataFrame.from_records(univtown_list)
    univtown_df.columns = ['State', 'RegionName']
    return univtown_df

In [54]:
univtown_df = get_list_of_university_towns()

In [55]:
def get_gdp_df():
    '''this function reads the gdp data and returns a dataframe with only the required columns'''
    gdplev_df = pd.ExcelFile('gdplev.xls')
    gdplev_df = gdplev_df.parse("Sheet1", skiprows=219)
    gdplev_df = gdplev_df[['1999q4', 9926.1]]
    gdplev_df.columns = ['Quarter','GDP']
    return gdplev_df

In [56]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    #your code here
    gdplev_df = get_gdp_df()
    for i in range(2, len(gdplev_df)):
        if (gdplev_df.iloc[i-2][1] > gdplev_df.iloc[i-1][1]) and (gdplev_df.iloc[i-1][1] > gdplev_df.iloc[i][1]):
            return gdplev_df.iloc[i-2][0]

In [57]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
    #your code here
    gdplev_df = get_gdp_df()
    start = get_recession_start()
    start_index = gdplev_df[gdplev_df['Quarter'] == start].index.tolist()[0]
    gdplev_df = gdplev_df.iloc[start_index:]
    for i in range(2, len(gdplev_df)):
        if (gdplev_df.iloc[i-2][1] < gdplev_df.iloc[i-1][1]) and (gdplev_df.iloc[i-1][1] < gdplev_df.iloc[i][1]):
            return gdplev_df.iloc[i][0]

In [58]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    #your code here
    gdp_df = get_gdp_df()
    gdp_start = gdp_df.index[gdp_df['Quarter'] == get_recession_start()][0]
    gdp_end = gdp_df.index[gdp_df['Quarter'] == get_recession_end()][0]
    return gdp_df.iloc[gdp_df.iloc[gdp_start:gdp_end+1, 1 ].idxmin(), 0]

In [59]:
def new_col_names():
    #generating the new coloumns names 
    years = list(range(2000,2017))
    quars = ['q1','q2','q3','q4']
    quar_years = []
    for i in years:
        for x in quars:
            quar_years.append((str(i)+x))
    return quar_years[:67]

def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    #your code here
    housing_df = pd.read_csv('City_Zhvi_AllHomes.csv', encoding='cp1252')
    housing_df['State'] = housing_df['State'].map(states)
    housing_df.set_index(['State', 'RegionName'], inplace=True)
    housing_df = housing_df.loc[:, '2000-01': ]
    new_columns = [str(x)+y for x in range(2000, 2017) for y in ['q1', 'q2', 'q3', 'q4']]
    new_columns = new_columns[:-1]
    x = 0

    for c in new_columns:
        housing_df[c] = housing_df.iloc[:, x:x+3].mean(axis=1)
        x = x+3
    housing_df = housing_df.loc[:, '2000q1':]
    return housing_df

In [60]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    #your code here
    start = get_recession_start()
    bottom = get_recession_bottom()
    housing = convert_housing_data_to_quarters()
    housing = housing.loc[:, start: bottom]
    housing.reset_index(inplace=True)
    housing['price_ratio'] = (housing[start] - housing[bottom]) / housing[start]
    univtown_df = get_list_of_university_towns()  
    univtown_df_list = univtown_df['RegionName'].tolist()
    housing['isUniTown'] = housing.RegionName.apply(lambda x: x in univtown_df_list)

    univtown = housing[housing.isUniTown].copy().dropna()
    not_univtown = housing[~housing.isUniTown].copy().dropna()

    p = ttest_ind(univtown['price_ratio'], not_univtown['price_ratio'])[1]
    different = p < 0.01   
    lower = 'university town' if univtown['price_ratio'].mean() < not_univtown['price_ratio'].mean() else 'non-university town'

    return (different, p, lower)

   

In [61]:
run_ttest()

(True, 1.5248978583060365e-05, 'university town')